## init

Since I am responsible for (virtually) all data updates to the database for `<plan name>`, there can be ~60+ logs each week to process. Some can be closed after the database has been updated, while others need to be re-assigned for further processing (mainly death cases). 

The update process has been more or less 'sufficiently' streamlined. Manually closing/re-assigning ~60 logs at once was a hassle, hence this script was created. It's not the fastest but it works error-free and allows me to do other things while it runs in the background. 

In [1]:
import os
import time

import pandas as pd
import yaml

# some of these aren't necessary but I'll leave it for now
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

### fxs

In [2]:
def return_yaml_data(
    key: str = None, file_path: str = os.getcwd() + "\\" + "config.yaml"
):
    with open(file_path, "r") as f:
        data = yaml.safe_load(f.read())

    if key is None:
        return data

    try:
        result = data[key]
    except KeyError:
        return data

    return result


def search_for_xpath_elem(xpath_str, time_limit=30):
    """While len of search list is 0, keep searching. If exceeds time limit,  returns empty list.

    Args:
        xpath_str (str): raw string - xpath
        time_limit (int, optional): Time limit in seconds. Defaults to 30.

    Returns:
        list: search results
    """

    ctr = 0
    t_search = driver.find_elements(By.XPATH, xpath_str)

    while (len(t_search) == 0) and (ctr < time_limit):
        time.sleep(1)
        ctr += 1
        t_search = driver.find_elements(By.XPATH, xpath_str)

    if len(t_search) > 0:
        return t_search
    else:
        return []

### login

In [3]:
# private
USERNAME = return_yaml_data("user")
PASSWORD = return_yaml_data("pass")
HARMONY_URL = return_yaml_data("url")
ASSIGN_TO = return_yaml_data("colleague")


serv = Service(return_yaml_data("chromedriver"))
driver = webdriver.Chrome(service=serv)


# goto Harmony
driver.get(HARMONY_URL)

# enter username
t_search = search_for_xpath_elem(r'//*[@id="username"]')
t_search[0].send_keys(USERNAME)

# enter password
t_search = search_for_xpath_elem(r'//*[@id="password"]')
t_search[0].send_keys(PASSWORD)
time.sleep(1)
t_search[0].send_keys(Keys.RETURN)

## run

### close logs

In [ ]:
# load data
df = pd.read_csv("./data/automate_harmony.csv")

rel_cols = ["log_action", "ticket_num"]
df = df[rel_cols].copy()

# alert if missing ticket num
mask = df.ticket_num.isna()
if df[mask].shape[0] != 0:
    print("some have missing ticket nums")
else:
    pass

# filter for only logs to close
mask = df["log_action"].str.strip().str.lower() == "n"
logs_to_close = df[mask].copy()
logs_to_close = logs_to_close.reset_index(drop=True).copy()
len(logs_to_close)

In [ ]:
completed_list = []

In [ ]:
for i in range(len(logs_to_close)):
    ticket_num = logs_to_close.loc[i, "ticket_num"]
    print(i)
    print(ticket_num)
    print(f"{i+1}/{len(logs_to_close)}")

    if ticket_num in completed_list:
        print("already done")
        print("")
        pass
    else:
        print("")
        time.sleep(1)

        # --click `log search (by tkt number)` button--
        log_search_xpath = r'//*[@id="clientless-menu"]/div/div/div/ul/li[3]/a/i'
        t_search = search_for_xpath_elem(log_search_xpath)
        time.sleep(1)
        t_search[0].click()

        # --fix--
        refresh_btn_xpath = r'//*[@id="harmony-loader"]'
        t_search = search_for_xpath_elem(refresh_btn_xpath)
        time.sleep(1)
        t_search[0].click()

        # --find and input val in `ticket number input box`--
        tkt_num_input_xpath = r'//*[@id="cmLogSearch"]/form/div[1]/div/input'
        t_search = search_for_xpath_elem(tkt_num_input_xpath)
        time.sleep(1)
        t_search[0].send_keys(str(ticket_num))

        # --find and click `ticket number search button`--
        tkt_num_button_xpath = (
            r'//*[@id="cmLogSearch"]/form/div[2]/div/button[1]/span[1]'
        )
        t_search = search_for_xpath_elem(tkt_num_button_xpath)
        time.sleep(1)
        t_search[0].click()

        # there should only be one result
        result_actions_xpath = r'//*[@id="cmLogSearch"]/div[2]/div/div/div/cm2-result/div/table/tbody/tr/td[10]/div/button/a'
        t_search = search_for_xpath_elem(result_actions_xpath)
        time.sleep(1)

        # drop down menu - can either view log or re-assign
        t_search[0].click()

        view_log_btn_xpath = r'//*[@id="cmLogSearch"]/div[2]/div/div/div/cm2-result/div/table/tbody/tr/td[10]/div/ul/li[1]/a/button/span'
        t_search = search_for_xpath_elem(view_log_btn_xpath)
        time.sleep(1)
        t_search[0].click()

        log_edit_btn_xpath = r'//*[@id="navbar-state"]/div[2]/div[2]/div/div/div[2]/div[2]/span/button[2]'
        t_search = search_for_xpath_elem(log_edit_btn_xpath)
        time.sleep(1)
        t_search[0].click()

        # different types of resolution input boxes
        xpath_list = [
            r'//*[@id="idocument-answer"]',
            r'//*[@id="icall-answer"]',
            r'//*[@id="ocall-answer"]',
        ]

        t_search = []
        while len(t_search) == 0:
            for xpath_str in xpath_list:
                t_search = driver.find_elements(By.XPATH, xpath_str)
                time.sleep(0.5)
                if len(t_search) > 0:
                    t_search[0].send_keys("done")
                    break  # exit out of loop once element found

        # --click submit--
        # different types of submit buttons
        xpath_list = [
            r'//*[@id="navbar-state"]/div[2]/div[2]/div/div/div[2]/form/div[6]/button[1]/span[1]',
            r'//*[@id="navbar-state"]/div[2]/div[2]/div/div/div[2]/form/div[8]/button[1]/span[1]',
            r'//*[@id="navbar-state"]/div[2]/div[2]/div/div/div[2]/form/div[7]/button[1]/span[1]',
        ]

        t_search = []
        while len(t_search) == 0:
            for xpath_str in xpath_list:
                t_search = driver.find_elements(By.XPATH, xpath_str)
                if len(t_search) > 0:
                    break  # exit out of loop once element found
        t_search[0].click()

        # add to completed list
        completed_list.append(ticket_num)
        print(f"done {ticket_num}")

        # go back to log search screen
        log_search_xpath = r'//*[@id="clientless-menu"]/div/div/div/ul/li[3]/a/i'
        t_search = search_for_xpath_elem(log_search_xpath)
        time.sleep(1)
        t_search[0].click()

### reassign logs

In [ ]:
# load data
df = pd.read_csv("./data/automate_harmony.csv")

rel_cols = ["log_action", "ticket_num"]
df = df[rel_cols].copy()

# alert if missing ticket num
mask = df.ticket_num.isna()
if df[mask].shape[0] != 0:
    print("some have missing ticket nums")
else:
    pass

# filter for only logs to re-assign
mask = df["log_action"].str.strip().str.lower() != "n"
logs_to_close = df[mask].copy()  # ignore this variable name
logs_to_close = logs_to_close.reset_index(drop=True).copy()
len(logs_to_close)

In [ ]:
# --goto ticker number input--
# click `log search (by tkt number)` button
log_search_xpath = r'//*[@id="clientless-menu"]/div/div/div/ul/li[3]/a/i'
t_search = search_for_xpath_elem(log_search_xpath)
time.sleep(0.5)
t_search[0].click()

# fix
refresh_btn_xpath = r'//*[@id="harmony-loader"]'
t_search = search_for_xpath_elem(refresh_btn_xpath)
time.sleep(1)
t_search[0].click()

In [ ]:
completed = []

In [ ]:
for i in range(len(logs_to_close)):
    ticket_num = logs_to_close.loc[i]["ticket_num"]
    note = logs_to_close.loc[i]["log_action"]

    if ticket_num in completed:
        print(i)
        print(ticket_num)
        print("already done")
        print("")
        pass

    else:
        print(i)
        print(ticket_num)
        print(f"{i+1}/{len(logs_to_close)}")
        print("")
        print(note)

        # --find and input val in `ticket number input box`--
        tkt_num_input_xpath = r'//*[@id="cmLogSearch"]/form/div[1]/div/input'
        t_search = search_for_xpath_elem(tkt_num_input_xpath)
        time.sleep(0.5)
        t_search[0].send_keys(str(ticket_num))

        # --find and click `ticket number search button`--
        tkt_num_button_xpath = (
            r'//*[@id="cmLogSearch"]/form/div[2]/div/button[1]/span[1]'
        )
        t_search = search_for_xpath_elem(tkt_num_button_xpath)
        time.sleep(0.5)
        t_search[0].click()

        # there should only be one result
        result_actions_xpath = r'//*[@id="cmLogSearch"]/div[2]/div/div/div/cm2-result/div/table/tbody/tr/td[10]/div/button/a'
        t_search = search_for_xpath_elem(result_actions_xpath)
        time.sleep(0.5)
        t_search[0].click()

        # click re-assign button
        re_assign_btn_xpath = r'//*[@id="cmLogSearch"]/div[2]/div/div/div/cm2-result/div/table/tbody/tr/td[10]/div/ul/li[2]/a/button/span'
        t_search = search_for_xpath_elem(re_assign_btn_xpath)
        time.sleep(0.5)
        t_search[0].click()

        # enter name in 'search user' input box
        search_user_xpath = r'//*[@id="basic-query"]'
        t_search = search_for_xpath_elem(search_user_xpath)
        time.sleep(0.5)
        t_search[0].send_keys(ASSIGN_TO)
        time.sleep(0.7)
        t_search[0].send_keys(Keys.RETURN)

        # click assign button
        assign_btn_xpath = r'//*[@id="top"]/div[1]/div/div/div/div[2]/div[2]/table/tbody/tr/td[1]/div/button'
        t_search = search_for_xpath_elem(assign_btn_xpath)
        time.sleep(0.5)
        t_search[0].click()

        # enter reason
        reason_input_xpath = r'//*[@id="reassign-reason"]'
        t_search = search_for_xpath_elem(reason_input_xpath)
        time.sleep(0.5)
        t_search[0].send_keys(note)

        time.sleep(1.5)

        # submit reason
        reason_submit_btn_xpath = (
            r'//*[@id="top"]/div[1]/div/div/div/div[3]/button[1]/span[1]'
        )
        t_search = driver.find_elements(By.XPATH, reason_submit_btn_xpath)
        t_search[0].click()

        time.sleep(3)

        completed.append(ticket_num)
        print(f"done {ticket_num}")

        # click reset form button
        reset_form_btn_xpath = r'//*[@id="cmLogSearch"]/form/div[2]/div/button[2]/span'
        t_search = driver.find_elements(By.XPATH, reset_form_btn_xpath)
        t_search[0].click()